# AI4All Fragile Families Final Project: Predicting GPA at Age 15

## GPA (taken from the[ Fragile Families Challenge Blog](http://www.fragilefamilieschallenge.org/gpa/))
GPA measures academic achievement.

We want to know:

- What helps disadvantaged children to beat the odds and succeed academically?
- What derails children so that they perform unexpectedly poorly?

This is the question that the 15 year olds were asked:
![Survey GPA Question](http://www.fragilefamilieschallenge.org/wp-content/uploads/2017/03/GPA_questionnaire-768x234.png)

### How was the data cleaned?
The measure of GPA is self-reported by the child at approximately age 15. They marked as NA the GPAs of children who were not interviewed, reported no grade, refused to answer, did not know, or were homeschooled, for any of the four subjects. For children with valid answers, we averaged the responses for all four subjects, then subtracted this number from 5 to produce an estimate of child GPA ranging from 1 to 4. In the re-coded variable, a GPA of 4.0 indicates that the child reported straight As, while a GPA of 1.0 indicates that the child reported getting all grades of D or lower.

### Motivation
Helping kids “beat the odds” academically is a fundamental goal of education research; academic success can be the key to breaking the cycle of poverty. Free public education is often referred to as a great equalizer, yet children who grow up in disadvantaged families consistently underperform their more affluent peers on average.

However, the average is not the whole story. Some kids do well despite being expected to do poorly. In fact, the amount of unexplained variation in educational achievement is enormous: **social science models typically have R-squared values of 0.2 or less.** The poor predictive performance of social science models of educational attainment has long been known. In the now-classic 1972 book Inequality: A Reassessment of the Effect of Family and Schooling in America, Harvard social scientist Christopher Jencks argued that random chance played a larger role than measured family background characteristics in determining socioeconomic outcomes.

While social scientists have learned some about what helps children succeed academically in the decades since 1972, a huge proportion of the variance remains buried in the error term of regression models. Is this term truly random chance, or is there “dark matter” out there in the form of unmeasured but important variables that help some kids to beat the odds?

# Why your model matters! 
By submitting a model for GPA at age 15, you help the Fragile Family researchers in their quest to find this dark matter. As you heard from professors Matt Salganik and Sara McLanahan, they have used the models from the Challenge to identify a subset of children performing much better and worse than expected. They are currently interviewing these children to answer the question: what unmeasured variables are common to the kids who are beating the odds, which we do not observe among the children who are struggling unexpectedly?

When you participate, you help them to better understand what affects the academic success of kids your age. **YOU more than your intructors or the Fragile Family professors are better-positioned to uncover unmeasured but important factors. It does not matter how well your model does. What matters is the insight you provide when thinking about you think affects kids your age.** Your models and the interviews will help researchers learn how some kids beat the odds. These results may drive future deductive research to evaluate the causal effect of these unmeasured variables, and ultimately we hope that policymakers can intervene on the “dark matter” we find in order to improve the lives of other disadvantaged children in the future.

# The Data

The Fragile Families Challenge is based on the Fragile Families and Child Wellbeing Study, which has followed thousands of American families for more than 15 years.  During this time, the Fragile Families study collected information about the children, their parents, their schools, and their larger environments.

These data have been used in hundreds of scientific papers and dozens of dissertations, and insight from these studies are routinely shared with policy makers around the world through the Future of Children, which is jointly published by Princeton University and Brookings Institution. Your challenge is to use this data in a new way. 

**Given all the background data from birth to year 9 and some training data from year 15, how well can you predict the GPA outcome in the year 15 test data?**

![Fragile Families Challenge Data](http://www.fragilefamilieschallenge.org/wp-content/uploads/2017/02/ff_design_matrix_challenge.png)

# Let's start! :)

Remember the main steps in building and testing a machine learning model:

1) [Load the Python libraries](#1.-Load-the-Libraries)

2) [Load the data](#2.-Load-the-Data)

3) [Do data imputation (data cleaning) by removing and or changing the missing values in the data](#3.-Data-Imputation)

4) [Explore and Visualize the data through bar plots, histograms, and scatter plots](#4.-Data-Exploration-and-Visualization)

5) [Create new interesting predictors by combining different features (feature engineering)](#5.-Feature-Engineering)

6) [Define your predictors and your outcome and split the data into training and testing sets](#6.-Train-Test-Split)

7) [Build and train your model using the training data](#7.-Model-Building-and-Training)

8) [Test the performance of the model using the testing data and an appropriate performance metric](#8.-Model-Testing)

9) [Go back to step 5 and improve your model!](#9.-Improving-the-model)

10) [Interpret the model results by examining the model coefficients and the performance metric](#10.-Interpreting-the-Results)

# 1. Load the Libraries

In [ ]:
# First, we load the libraries we will need for this notebook
%matplotlib inline 
import matplotlib.pyplot as plt
import scipy.stats as stats # this library provides functions for performing statistics on data
import pandas as pd
import numpy as np
import sys
import sklearn # this library provides functions for performing machine learning on data
import ff_functions as fff # this is the library we are providing for you to perform common operations on data

# The following line mutes a Pandas warning that we don't need to worry about at the moment
pd.options.mode.chained_assignment = None  # default='warn'

# 2. Load the Data

In [ ]:
# We read in the Fragile Families data as in previous notebooks

# First, we tell Python the location of the folder with the background.cvs data file
background = "../ff_data/background.csv"
# What does this line do?
data_frame = pd.read_csv(background, low_memory=False)
# We set the index of the DataFrame to match the challengeID
data_frame = data_frame.set_index('challengeID')
data_frame.head()

In [ ]:
# Let's read in the GPA training data for year 15. This is in a different file called train.csv

# First, we tell Python the location of the folder with the train.cvs data file
train = "../ff_data/train.csv"
# What does this line do?
data_frame_outcomes = pd.read_csv(train, low_memory=False)
# We set the index of the DataFrame to match the challengeID
data_frame_outcomes = data_frame_outcomes.set_index('challengeID')
data_frame_outcomes.head()

# 3. Data Imputation

In [ ]:
# Let's print a summary of the GPA data
data_frame_outcomes['gpa'].describe()

In [ ]:
# How many rows do we have?
nrows = data_frame_outcomes.shape[0]
print("We have ",nrows," rows.")
# How many missing values do we have in the data?
nmissing = data_frame_outcomes['gpa'].isnull().sum()
print("We have ",nmissing," rows are NaN.")
print("We have ",nrows-nmissing," rows are not NaN.")
percentMissing = (nmissing/nrows)*100;
print(str(percentMissing)+'% of the rows are NaN.')

In [ ]:
# Imputation method 1: remove rows with NaN
gpa = fff.pick_ff_variables(data_frame_outcomes,['gpa'],remove_nans=True)
gpa.head()

In [ ]:
# Imputation method 2: replace NaN with minimum value of 1 
# Why would we do this?

#gpa = pd.DataFrame(data_frame_outcomes['gpa'].fillna(1))
#gpa.head()

In [ ]:
# Imputation method 3: replace NaN with average gpa value 
# Why would we do this?

#avg_gpa = data_frame_outcomes['gpa'].mean()
#gpa = pd.DataFrame(data_frame_outcomes['gpa'].fillna(avg_gpa))
#gpa.head()

# 4. Data Exploration and Visualization

In [ ]:
# Plot a histogram of GPA
fff.plot_histogram(gpa['gpa'],labels=['gpa'],xlabel='GPA',ylabel='count',title='GPA Histogram')

In [ ]:
gpa1_2 = gpa[gpa['gpa']<2].shape[0]
gpa2_3 = gpa[(gpa['gpa']>=2)&(gpa['gpa']<3)].shape[0]
gpa3_4 = gpa[gpa['gpa']>=3].shape[0]

fff.one_pie_chart([gpa1_2,gpa2_3,gpa3_4],label='GPA',pie_labels=['Between 1 and 2','Between 2 and 3','Between 3 and 4'])

# 5. Feature Engineering

We will provide you with a few features that will help you get started. These features are:

1) Mother's race (cm1ethrace)

2) Mother's level of education (cm1edu)

3) Mother's marriage status (cm1relf)

4) Teacher's evaluation of student's skills in Year 9 (engineered feature: average of t5c13a, t5c13b, and t5c13c)

In [ ]:
df = fff.pick_ff_variables(data_frame,['cm1ethrace','cm1edu','cm1relf','t5c13a','t5c13b','t5c13c'],remove_nans=True,remove_negatives=True)
# we also simplify the answers in cm1relf to consider only three cases: married, cohabitating, and other
df.loc[df['cm1relf']>=3,'cm1relf']=3

df.head()

Here we create a new feature to predict GPA based on the teacher's evaluation of the student's skills in Year 9. We will average the following 3 features:

1) Teacher's evaluation of child's Language and Literacy Skills ('t5c13a')

2) Teacher's evaluation of child's Science and Social Studies skills ('t5c13b')

3) Teacher's evaluation of child's Math skills ('t5c13c')

In [ ]:
df['skills'] = df[['t5c13a','t5c13b','t5c13c']].sum(axis=1)/3
df.head()

## One-Hot Encoding of Categorical Variables

Remember that to use categorical variables in linear regression, we need to do one-hot encoding which creates a number of binary columns, each per each category.

In [ ]:
# One-hot encoding of race
race = pd.get_dummies(df['cm1ethrace'])
race.columns = ['White','Black','Hispanic','Other Race']
race.head()

In [ ]:
# One-hot encoding of level of education
education = pd.get_dummies(df['cm1edu'])
education.columns = ['Less than HS','HS','Some College/Tech','College/Grad']
education.head()

In [ ]:
# One-hot encoding of relationship to father
relationship = pd.get_dummies(df['cm1relf'])
relationship.columns = ['Married','Cohabitating','Other Rel']
relationship.head()

### Combine all one-hot encoded columns

In [ ]:
predictors = pd.concat([df['skills'],race,education,relationship],axis=1)
predictors.head()

# 6. Train Test Split

Before we can split the data into training and testing data sets, we need to make sure that the GPA data frame has the same rows as the predictors data frame

In [ ]:
data = pd.merge(gpa, predictors, left_index=True, right_index=True)
data.head()

In [ ]:
from sklearn.model_selection import train_test_split

# The following lines of code randomly splits both the outcome data and the predictor data into 
# a training and a testing set. The test_size argument is a number between 0 and 1 that tells the 
# function the percentage of data that should go into the test set.

x = data.iloc[:,1:] # predictors are all columns of data except column 0
y = data['gpa'] # outcome is the first column of data

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3, random_state=100)

print(df.shape)
print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

# Transform to numpy arrays
train_x = fff.pandas_to_2d_numpy(train_x)
train_y = fff.pandas_to_2d_numpy(train_y)

test_x = fff.pandas_to_2d_numpy(test_x)
test_y = fff.pandas_to_2d_numpy(test_y)

# 7. Model Building and Training

In [ ]:
# import the linear_model function from sklearn
from sklearn import linear_model

# intialize the model
reg = linear_model.LinearRegression()

# fit the model ON THE TRAINING DATA (important!)
reg.fit(train_x, train_y)

In [ ]:
# Print the trained model coefficients

# c0 is eual to the model intercept
print("c0 = ",reg.intercept_)

# the other coefficients tells us how the features affect the outcome
i=1
for c in reg.coef_[0]:
    print('c'+str(i)+'=',c)
    i+=1

# 8. Model Testing

In [ ]:
# How well can the model predict the testing data?
pred_y = reg.predict(test_x)
# Now, we can quantify how good our model is at predicting new values by computing R2 score
R2 = sklearn.metrics.r2_score(test_y, pred_y)
print('mean root of square error : {}'.format(R2))

In [ ]:
# How well can the model predict the training data? (sanity check)
pred_y = reg.predict(train_x)
# Now, we can quantify how good our model is at predicting new values by computing R2 score
R2 = sklearn.metrics.r2_score(train_y, pred_y)
print('mean root of square error : {}'.format(R2))

# 9. Improving the model

- Are there any more interesting features in the data that you think might affect GPA?
- Are there any features that you can create with the data that you think might be a good predictor for GPA?
- Have you tried a different imputation method?
    - Removing missing values
    - Replacing missing values with the average of the column
    - Replacing missing values with the median of the column
    - Replacing missing values with another statistic that makes sense

# 10. Interpreting the Results